In [1]:
import pandas as pd
import os
from googleapiclient.discovery import build
from IPython.display import JSON
import isodate
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, text, ForeignKey

### YouTube API credentials

In [2]:
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")

In [3]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

### Import channel IDs from CSV

In [4]:
# File path for CSV import
csv_file_path = 'channels/channel_ids.csv'

# Read channel IDs from CSV file into a list
df = pd.read_csv(csv_file_path, header=None)
channel_ids = df[0].tolist()

### Function to return channel details

In [5]:
def get_channel_stats(youtube, channel_ids):
    all_data = []

    try:
        # Split channel_ids list into chunks of up to 50 ids each
        id_chunks = [channel_ids[i:i+50] for i in range(0, len(channel_ids), 50)]

        for id_chunk in id_chunks:
            request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(id_chunk)
            )
            response = request.execute()

            for item in response.get('items', []):
                data = {
                    'channel_name': item['snippet']['title'],
                    'channel_id': item['id'],
                    'subscriber_count': item['statistics']['subscriberCount'],
                    'view_count': item['statistics']['viewCount'],
                    'video_count': item['statistics']['videoCount'],
                    'playlist_id': item['contentDetails']['relatedPlaylists']['uploads'],
                    'start_date': item['snippet']['publishedAt'],
                    'country': item['snippet']['country'],  
                }
                all_data.append(data)
    except Exception as e:
        print(f"Error occurred: {e}")

    return pd.DataFrame(all_data)

### Get channel data

In [6]:
channels_df = get_channel_stats(youtube, channel_ids)

### Covert start date to dt format

In [7]:
channels_df['start_date'] = pd.to_datetime(channels_df['start_date']).dt.date

In [8]:
channels_df.head()

,channel_name,channel_id,subscriber_count,view_count,video_count,playlist_id,start_date,country
0,Exaltitude,UCelp2mfpa0fors7HePAodoA,17200,771666,64,UUelp2mfpa0fors7HePAodoA,2021-05-18,US
1,NeuralNine,UC8wZnXYK_CGKlBcZp-GxYPA,304000,22938962,695,UU8wZnXYK_CGKlBcZp-GxYPA,2019-04-04,AT
2,Simple LEARN,UCUb5LzgHU7nXT9Dbpt-EQjw,2,69,2,UUUb5LzgHU7nXT9Dbpt-EQjw,2023-02-20,IN
3,Dave Ebbelaar,UCn8ujwUInbJkBhffxqAPBVQ,59500,2133127,62,UUn8ujwUInbJkBhffxqAPBVQ,2012-07-01,NL


## Export channel data to database

In [45]:
db_string = 'sqlite:///database/youtube.db'

# Create a engine
engine = create_engine(db_string)

# Create a MetaData object
metadata_obj = MetaData()

# Define the 'channel' table
channel_table = Table(
    'channel',
    metadata_obj,
    Column('channel_id', String, primary_key=True),
    Column('channel_name', String),
    Column('subscriber_count', Integer),
    Column('view_count', Integer),
    Column('video_count', Integer),
    Column('playlist_id', String),
    Column('start_date', DateTime),
    Column('country', String),
)

In [46]:
# Create connection
conn = engine.connect()

# Create all tables in the database
metadata_obj.create_all(engine)

In [47]:
# Push df to database
channels_df.to_sql("channel", engine, if_exists='append', index=False)

4

In [35]:
# Test database with simple query
query = text('SELECT * FROM channel')
test_df = pd.read_sql_query(query, conn)
test_df.head()

,channel_id,channel_name,subscriber_count,view_count,video_count,playlist_id,start_date,country
0,UCUb5LzgHU7nXT9Dbpt-EQjw,Simple LEARN,2,69,2,UUUb5LzgHU7nXT9Dbpt-EQjw,2023-02-20,IN
1,UCn8ujwUInbJkBhffxqAPBVQ,Dave Ebbelaar,59500,2133127,62,UUn8ujwUInbJkBhffxqAPBVQ,2012-07-01,NL
2,UC8wZnXYK_CGKlBcZp-GxYPA,NeuralNine,304000,22938962,695,UU8wZnXYK_CGKlBcZp-GxYPA,2019-04-04,AT
3,UCelp2mfpa0fors7HePAodoA,Exaltitude,17200,771666,64,UUelp2mfpa0fors7HePAodoA,2021-05-18,US


### Get all video ids from a channel

In [14]:
def get_video_ids(youtube, playlist_id):

    video_ids = []

    try:
        # Initial request for the first page of videos
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50
        )
        response = request.execute()

        # Extract video IDs from the first page of response
        for item in response.get('items', []):
            video_ids.append(item['contentDetails']['videoId'])

        # Fetch additional pages of videos, if available
        while 'nextPageToken' in response:
            request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=response['nextPageToken']
            )
            response = request.execute()

            # Extract video IDs from the current page of response
            for item in response.get('items', []):
                video_ids.append(item['contentDetails']['videoId'])
    except Exception as e:
        print(f"Error occurred: {e}")

    return video_ids

### Get video details for video IDs

In [15]:
def get_video_details(youtube, video_ids):

    all_video_info = []

    try:
        # Split video_ids list into chunks of up to 50 IDs each
        id_chunks = [video_ids[i:i+50] for i in range(0, len(video_ids), 50)]

        for id_chunk in id_chunks:
            request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=','.join(id_chunk)
            )
            response = request.execute()

            for video in response.get('items', []):
                video_info = {
                    'channel_id': video['snippet']['channelId'],
                    'video_id': video['id'],
                    'video_title': video['snippet']['title'],
                    'description': video['snippet']['description'],
                    'tags': video['snippet'].get('tags', []),
                    'published': video['snippet']['publishedAt'],
                    'view_count': video['statistics'].get('viewCount', None),
                    'like_count': video['statistics'].get('likeCount', None),
                    'favourite_count': video['statistics'].get('favoriteCount', None),
                    'comment_count': video['statistics'].get('commentCount', None),
                    'duration': video['contentDetails'].get('duration', None),
                    'definition': video['contentDetails'].get('definition', None),
                    'caption': video['contentDetails'].get('caption', None),
                    'category_id': video['snippet'].get('categoryId', None),
                }
                all_video_info.append(video_info)
    except Exception as e:
        print(f"Error occurred: {e}")

    return pd.DataFrame(all_video_info)


### Get video data for each channel

In [16]:
videos_df = pd.DataFrame()

for channel in channels_df['channel_name']:
    # Get playlist id for channel
    playlist_id = channels_df.loc[channels_df['channel_name'] == channel, 'playlist_id'].iloc[0]
    # Get all video ids for channel
    video_ids = get_video_ids(youtube, playlist_id)
    # Get video data for each video
    video_data = get_video_details(youtube, video_ids)

    # Concat to main df
    videos_df = pd.concat([videos_df, video_data], ignore_index=True)

### Convert dates to dt

In [17]:
videos_df['published'] = pd.to_datetime(videos_df['published'])

### Covert duration to seconds using ISODATE

In [18]:
def duration_to_seconds(duration_str):
    duration = isodate.parse_duration(duration_str)
    total_seconds = duration.total_seconds()
    return int(total_seconds)

In [19]:
videos_df['duration'] = videos_df['duration'].apply(duration_to_seconds)

### Drop videos less than 120 and more than 1800 seconds

In [20]:
videos_df = videos_df[(videos_df['duration'] >= 60) & (videos_df['duration'] <= 1800)]
# Reset the index
videos_df = videos_df.reset_index(drop=True)

### Convert tag list to string

In [21]:
videos_df['tags'] = videos_df['tags'].apply(', '.join)

In [41]:
videos_df.tail()

,channel_id,video_id,video_title,description,tags,published,view_count,like_count,favourite_count,comment_count,duration,definition,caption,category_id
709,UCelp2mfpa0fors7HePAodoA,yaaa7U3qmL0,Secret to Landing a Software Engineering Job,📎 Resources\n==============\n✅ FREE ATS-Friend...,"How to land a dream job, Software engineer dre...",2023-05-02 01:00:33+00:00,856,57,0,1,76,hd,false,27
710,UCelp2mfpa0fors7HePAodoA,4Amf5FsnvOQ,LinkedIn Cold Outreach Template: Job Hunting a...,"Template:\nHi [Name], I came across your profi...","Cold outreach, Exaltitude, Job search tips, Li...",2023-04-30 17:29:20+00:00,1130,85,0,8,60,hd,false,27
711,UCelp2mfpa0fors7HePAodoA,4Dhc-Ji4O08,Which Software Engineer Job Gets Highest Salary,📎 Resources\n==============\n✅ The full fyi re...,"Best paying tech employers, Exaltitude, High p...",2023-04-19 17:24:14+00:00,2960,146,0,10,619,hd,false,27
712,UCelp2mfpa0fors7HePAodoA,UnT0iQkZJvE,Resume Review from your Hiring Manager,Resources\n==============\n👉 FREE ATS-Friendly...,"Application tracking system, Junior software e...",2023-04-05 21:49:11+00:00,3866,161,0,6,515,hd,false,27
713,UCelp2mfpa0fors7HePAodoA,Rf5tJna0CPU,Can Your Engineering Resume Get You Noticed? H...,Download the resume checklist - https://mailch...,"Application tracking system, applicant trackin...",2021-06-01 20:21:55+00:00,638,11,0,2,492,hd,false,22


### Export video df to csv

In [48]:
# Define the 'video' table
video_table = Table(
    'video',
    metadata_obj,
    Column('channel_id', ForeignKey("channel.channel_id")),
    Column('video_id', String, primary_key=True),
    Column('video_title', String),
    Column('description', String),
    Column('tags', String),
    Column('published', DateTime),
    Column('view_count', Integer),
    Column('like_count', Integer),
    Column('favourite_count', Integer),
    Column('comment_count', Integer),
    Column('duration', Integer),
    Column('definition', String),
    Column('caption', String),
    Column('category_id', Integer),
)

# Create all tables in the database
metadata_obj.create_all(engine)

In [49]:
# Push df to database
videos_df.to_sql(name="video", con=engine, if_exists='append', index=False)

714

In [43]:
# Test database with simple query
query = text("SELECT * FROM video")
test_df = pd.read_sql_query(query, conn)
test_df.head()

,channel_id,video_id,video_title,description,tags,published,view_count,like_count,favourite_count,comment_count,duration,definition,caption,category_id
0,UCUb5LzgHU7nXT9Dbpt-EQjw,w-tJy1Y71Zs,अणुभार की परिभाषा। अणुभार कैस ज्ञात करें। How ...,"Hello friends \nऐसे ही basic information, basi...",#Simplelearn,2023-02-21 13:43:00.000000,29,4,0,0,125,hd,false,27
1,UCn8ujwUInbJkBhffxqAPBVQ,H6kKmMB-LdQ,Introducing ClickUp Brain (this is huge for devs),"In this video, we dive into ClickUp's new ""Bra...","data science, python, machine learning, vscode...",2024-01-27 15:45:19.000000,1985,72,0,18,533,hd,false,27
2,UCn8ujwUInbJkBhffxqAPBVQ,OEPte4cG3Fk,Easiest Way to Connect AI Chatbots to WhatsApp,Let's build an AI chatbot with Botpress and co...,"data science, python, machine learning, vscode...",2024-01-17 14:00:15.000000,4500,162,0,20,1602,hd,false,27
3,UCn8ujwUInbJkBhffxqAPBVQ,_DAG-OH0wyc,All The Apps I Use On My New M3 MacBook (as a ...,Let's go over all the apps that I installed on...,"data science, python, machine learning, vscode...",2024-01-06 19:21:37.000000,2765,99,0,22,1749,hd,false,27
4,UCn8ujwUInbJkBhffxqAPBVQ,Ff3tJ4pJEa4,PostgreSQL as VectorDB - Beginner Tutorial,"In this video, I explain why I recently switch...","data science, python, machine learning, vscode...",2023-12-21 15:57:34.000000,4417,191,0,28,865,hd,false,27


In [44]:
# Close the connection
conn.close()
engine.dispose()